Source: https://python.langchain.com/docs/tutorials/llm_chain/

In [1]:
# Load the saved API keys

import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 44, 'total_tokens': 48, 'completion_time': 0.000719041, 'prompt_time': 0.01801412, 'queue_time': 0.04583666, 'total_time': 0.018733161}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b905f64f-b15a-4d3b-82c8-5deb8f190fdb-0', usage_metadata={'input_tokens': 44, 'output_tokens': 4, 'total_tokens': 48})

## Exercise
Practice tweaking the above cells with different models, different SYSTEM messages, different Human messages

In [7]:
messages = [
    SystemMessage("Translate the following from English into Dutch"),
    HumanMessage("Simply Simply Lovely"),
]

model.invoke(messages)

AIMessage(content='Eenvoudig, Eenvoudig, Prachtig', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 45, 'total_tokens': 59, 'completion_time': 0.041288323, 'prompt_time': 0.011219328, 'queue_time': 0.050263492, 'total_time': 0.052507651}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e8e50c94-a672-4558-bac7-634517fa97ad-0', usage_metadata={'input_tokens': 45, 'output_tokens': 14, 'total_tokens': 59})

In [8]:
messages = [
    SystemMessage("Given the information about three drivers, tell me which driver should I recruit as the main driver and which driver as the support driver for the 2025 season."),
    HumanMessage("Sebastian Vettel won the world championship from 2010 to 2013, Lewis Hamilton won the world championship in the years 2008, 2014, 2015, and 2017 to 2020, Max Verstappen won the world championship from 2021-2024"),
    HumanMessage("Which driver combination is the best?"),
]

model.invoke(messages)

AIMessage(content="Based on the provided information, I'd recommend recruiting Lewis Hamilton as the main driver and Max Verstappen as the support driver for the 2025 season.\n\nHere's the reasoning:\n- Lewis Hamilton has a proven track record of winning multiple championships (7 titles: 2008, 2014, 2015, and 2017-2020), showcasing his consistency and skill over the years.\n- Max Verstappen, on the other hand, has won the most recent championships (2021-2024), indicating that he is in top form and has a strong understanding of the current racing environment.\n\nBy recruiting Hamilton as the main driver, you'll benefit from his extensive experience and championship-winning mentality. Having Verstappen as the support driver will provide a strong backup and potentially create a competitive rivalry within the team, pushing both drivers to perform at their best.\n\nSebastian Vettel, although a highly accomplished driver, has not won a championship since 2013, which might indicate that he's 

# Streaming Message

In [9]:
import time
for token in model.stream(messages):
    print(token.content, end="|")
    time.sleep(0.1)

|Based| on| the| information| provided|,| here|'s| an| analysis| of| the| drivers|:

|1|.| **|Se|bastian| V|ettel|**:| A| four|-time| world| champion| with| consecutive| titles| from| |201|0| to| |201|3|.| However|,| his| last| championship| win| was| in| |201|3|,| which| is| over| |11| years| ago|.
|2|.| **|Lewis| Hamilton|**:| A| highly| decorated| driver| with| seven| world| championships|,| including| a| streak| of| four| consecutive| titles| from| |201|7| to| |202|0|.| His| last| championship| win| was| in| |202|0|,| which| is| more| recent| than| V|ettel|'s|.
|3|.| **|Max| Ver|st|appen|**:| The| current| dominant| force| in| the| sport|,| with| four| consecutive| world| championships| from| |202|1| to| |202|4|.| He| is| in| top| form| and| has| a| strong| recent| record|.

|Considering| the| recent| performance| and| championship| wins|,| I| would| recommend| the| following| driver| combination|:

|**|Main| Driver|:| Max| Ver|st|appen|**
|**|Support| Driver|:| Lewis| Hamilton|**


## Prompt templates

In [11]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

print(prompt_template)

input_variables=['language', 'text'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following from English into {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})]


In [12]:
prompt = prompt_template.invoke({"language": "Dutch", "text": "Yes! Oh my god!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Dutch', additional_kwargs={}, response_metadata={}), HumanMessage(content='Yes! Oh my god!', additional_kwargs={}, response_metadata={})])

In [13]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Dutch', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Yes! Oh my god!', additional_kwargs={}, response_metadata={})]

In [14]:
response = model.invoke(prompt)
print(response.content)

Ja! O mijn god!


# Structured Output

https://python.langchain.com/docs/how_to/structured_output/

In [15]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [16]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic outout structure
class JokeFormat(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(JokeFormat)

joke = structured_llm.invoke("Tell me a joke about cats")
print(joke)

setup='Why did the cat join a band?' punchline='Because it wanted to be a purr-cussionist' rating=8


In [17]:
#example

from typing import Optional
from pydantic import BaseModel, Field

# Pydantic
class AnimalFormat(BaseModel):
    """General description about a typical animal"""
    weight: int = Field(description="average weight of the animal in kg")
    lifeExpectancy: int = Field(description="Average Life expectancy of the animal in years")


structured_llm = llm.with_structured_output(AnimalFormat)

output = structured_llm.invoke("Tell me about human")
print(output)

weight=62 lifeExpectancy=72


In [19]:
# exercise: rag query

# Exercise: TOY RAG prompts

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

# RAG external text
file = open("./formula1.txt")
text = file.read()
#text = text[-len(text)//10:]
#print(text)

# Pydantic
class BookSummary(BaseModel):
    """A model to summarize a story book"""
    summary: str = Field(description="A short summary of the book")
    listOfCharacters: list[str] = Field(description="List of characters in the book")



# example toy RAG
prompt = ChatPromptTemplate([prompt_template])
msg = prompt.invoke({"question":"Summerize the book and give a list of main characters in the story.", "context":text})


structured_model = model.with_structured_output(BookSummary)

output = structured_model.invoke(msg)
print(output)

summary="The story revolves around the mysterious disappearance of the Veridian family from their old mansion a century ago. The protagonist, Elias Thorne, a historian, uncovers the truth through a journal belonging to the eldest daughter, Elara, and discovers that the family didn't vanish but became part of the garden, transforming into something timeless and beautiful. The main characters include Elias Thorne, Elara, Finn, and the rest of the Veridian family" listOfCharacters=['Elias Thorne', 'Elara', 'Finn', 'Mr. Veridian', 'Mrs. Veridian', 'Elderly Aunt']


In [20]:
# Exercise: Generate a list of question and answers based on a given text

prompt_template = """You are an assistant for a research scholar. Based on the given article, list major points of the article. Don't user prior knowledge.
Artile: {article} 
Major Points:"""

# RAG external text
file = open("./f1data.txt")
text = file.read()

# example toy RAG
prompt = ChatPromptTemplate([prompt_template])
msg = prompt.invoke({"article":text})

output = llm.invoke(msg)

In [21]:
print(output.content)

Based on the given article, the major points are:

1. **Introduction to Formula 1**: Formula 1 (F1) is the highest class of international single-seater auto racing, combining technology, strategy, and human skill.

2. **The F1 Car**: F1 cars are a blend of power and precision, with a strict set of rules that teams must follow, and they utilize hybrid power units and advanced aerodynamics.

3. **Hybrid Power Units**: F1 cars are powered by turbo-hybrid power units, which include a 1.6-liter turbocharged V6 engine and an Energy Recovery System (ERS) that recovers energy from braking and exhaust gases.

4. **Aerodynamics and Downforce**: The cars' performance is dependent on aerodynamics, with features like front and rear wings, underbody, and diffuser creating significant downforce, and the Drag Reduction System (DRS) aiding in overtaking.

5. **The Grand Prix Weekend**: A typical F1 race weekend is a three-day event, consisting of practice sessions on Friday, practice and qualifying on 

In [22]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./strategy.pdf")
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [23]:
pages[12].page_content

'COLLABORA TE\nCREA TE WIN\n-WIN RELA TIONSHIPS WITH OUR PARTNERS'

# More Bigger Prompting

In [24]:
question_template = """Tell me a joke about: {topic}
Write the response in the following format:

# {{Title}}
--------
{{response}}
"""
question = question_template.format(topic="formula 1")
question


'Tell me a joke about: formula 1\nWrite the response in the following format:\n\n# {Title}\n--------\n{response}\n'

In [25]:
response =  model.invoke(question_template.format(topic="formula 1"))

In [26]:
from IPython.display import Markdown
Markdown(response.content)

# Formula for Laughs
--------
Why did the Formula 1 driver bring a ladder to the track? Because he wanted to take his racing to the next level.

## Example 

In [27]:
prompt = """You are a research assistant. You are tasked to find 4 subtopics for the research area {topic}. For each sub area, list 5 major research work done so far in this format:

## {{Sub topic 1}}
1. {{Work 1}}
2. {{Work 2}}
3. {{Work 3}}
4. {{Work 4}}
5. {{Work 6}}
----
## {{Sub topic 2}}
1. {{Work 1}}
2. {{Work 2}}
3. {{Work 3}}
4. {{Work 4}}
5. {{Work 6}}
----
...
## {{Sub topic 4}}
1. {{Work 1}}
2. {{Work 2}}
3. {{Work 3}}
4. {{Work 4}}
5. {{Work 6}}
 
"""

response = model.invoke(prompt.format(topic="Quantum Computing for Machine Learning"))
from IPython.display import Markdown
from IPython.display import Markdown
Markdown(response.content)

## Quantum Machine Learning Algorithms
1. **Quantum k-Means**: A quantum algorithm for k-means clustering, which can potentially speed up the clustering process for large datasets.
2. **Quantum Support Vector Machines**: A quantum algorithm for support vector machines, which can potentially improve the classification accuracy for certain types of datasets.
3. **Quantum Neural Networks**: A quantum algorithm for neural networks, which can potentially speed up the training process for deep learning models.
4. **Quantum k-Nearest Neighbors**: A quantum algorithm for k-nearest neighbors, which can potentially improve the classification accuracy for certain types of datasets.
5. **Quantum Gradient Descent**: A quantum algorithm for gradient descent, which can potentially speed up the optimization process for machine learning models.

----
## Quantum Computing for Deep Learning
1. **Quantum Convolutional Neural Networks**: A quantum algorithm for convolutional neural networks, which can potentially speed up the training process for image classification tasks.
2. **Quantum Recurrent Neural Networks**: A quantum algorithm for recurrent neural networks, which can potentially speed up the training process for natural language processing tasks.
3. **Quantum Generative Adversarial Networks**: A quantum algorithm for generative adversarial networks, which can potentially improve the generation quality for certain types of datasets.
4. **Quantum Autoencoders**: A quantum algorithm for autoencoders, which can potentially improve the dimensionality reduction for certain types of datasets.
5. **Quantum Restricted Boltzmann Machines**: A quantum algorithm for restricted Boltzmann machines, which can potentially improve the feature learning for certain types of datasets.

----
## Quantum Dimensionality Reduction
1. **Quantum Principal Component Analysis**: A quantum algorithm for principal component analysis, which can potentially speed up the dimensionality reduction process for large datasets.
2. **Quantum t-Distributed Stochastic Neighbor Embedding**: A quantum algorithm for t-distributed stochastic neighbor embedding, which can potentially improve the visualization quality for high-dimensional datasets.
3. **Quantum Linear Discriminant Analysis**: A quantum algorithm for linear discriminant analysis, which can potentially improve the classification accuracy for certain types of datasets.
4. **Quantum Multidimensional Scaling**: A quantum algorithm for multidimensional scaling, which can potentially improve the visualization quality for high-dimensional datasets.
5. **Quantum Isomap**: A quantum algorithm for isomap, which can potentially improve the dimensionality reduction for certain types of datasets.

----
## Quantum Machine Learning for Computer Vision
1. **Quantum Image Classification**: A quantum algorithm for image classification, which can potentially improve the classification accuracy for certain types of images.
2. **Quantum Object Detection**: A quantum algorithm for object detection, which can potentially improve the detection accuracy for certain types of objects.
3. **Quantum Image Segmentation**: A quantum algorithm for image segmentation, which can potentially improve the segmentation accuracy for certain types of images.
4. **Quantum Image Generation**: A quantum algorithm for image generation, which can potentially improve the generation quality for certain types of images.
5. **Quantum Facial Recognition**: A quantum algorithm for facial recognition, which can potentially improve the recognition accuracy for certain types of faces.

## Example with sources

In [28]:
prompt_template = """You are an expert technical writer. 
            
Your task is to create a short report on the topic {focus}. Divide the report into easily digestable sections. You have to use prior knowledge. While writing the report use the following instructions:
        
1. Create a report structure using markdown formatting: Create a minimum of 4 sections.
- Use ## for the section title
- Use ### for sub-section headers
        
2. Write the report following this structure:
a. Title (## header)
b. Summary (### header)
c. Sources (### header)

3. Make your title engaging based upon the focus area of the analyst: 
{focus}

4. Write a summary section:
- Set up summary with general background / context related to the focus area of the analyst
- Emphasize what is novel, interesting, or surprising about insights gathered from the interview
- Create a numbered list of source documents, as you use them
- Do not mention the names of interviewers or experts
- Aim for approximately 2000 words maximum
- Use numbered sources in your report (e.g., [1], [2]) based on information from source documents
        
5. Write a Sources section:
- Include all sources used in your report
- Provide full links to relevant websites or specific document paths
- Separate each source by a newline. Use two spaces at the end of each line to create a newline in Markdown.
- It will look like:

### Sources
- [1] Link or Document name
- [2] Link or Document name

6. Be sure to combine sources. For example this is not correct:

[3] https://ai.meta.com/blog/meta-llama-3-1/
[4] https://ai.meta.com/blog/meta-llama-3-1/

There should be no redundant sources. It should simply be:

[3] https://ai.meta.com/blog/meta-llama-3-1/
        
7. Final review:
- Ensure the report follows the required structure
- Include no preamble before the title of the report
- Check that all guidelines have been followed

"""



response = model.invoke(prompt_template.format(focus="Neuromorphic Computing for Machine Learning"))
from IPython.display import Markdown
Markdown(response.content)

## Neuromorphic Computing for Machine Learning
### Summary
Neuromorphic computing is an emerging field that involves the development of computer systems that mimic the structure and function of the human brain. This field has gained significant attention in recent years due to its potential to revolutionize machine learning. Traditional computing systems are based on the von Neumann architecture, which is not well-suited for machine learning tasks that require parallel processing and adaptive learning. Neuromorphic computing, on the other hand, is inspired by the brain's neural networks and is designed to provide a more efficient and adaptive way of processing information [1].

The human brain is a complex and highly efficient system that is capable of processing vast amounts of information in parallel. It is estimated that the brain contains over 86 billion neurons, each of which can communicate with thousands of other neurons [2]. This complex network of neurons allows the brain to learn and adapt in a highly efficient manner, making it an ideal model for machine learning systems. Neuromorphic computing systems are designed to mimic this structure and function, using artificial neural networks to process information in a parallel and adaptive manner.

One of the key advantages of neuromorphic computing is its ability to provide real-time processing and learning. Traditional machine learning systems require large amounts of data to be processed in batches, which can be time-consuming and inefficient. Neuromorphic systems, on the other hand, can process information in real-time, allowing for faster and more adaptive learning [3]. This makes them ideal for applications such as robotics, autonomous vehicles, and smart homes, where real-time processing and learning are critical.

Some of the key technologies that are driving the development of neuromorphic computing include memristors, spintronics, and neuromorphic chips [4]. Memristors are a type of memory device that can store data in a non-volatile manner, allowing for more efficient and adaptive processing. Spintronics is a technology that uses the spin of electrons to process information, providing a more efficient and scalable way of processing data. Neuromorphic chips are specialized chips that are designed to mimic the structure and function of the brain, providing a highly efficient and adaptive way of processing information.

The potential applications of neuromorphic computing are vast and varied. Some of the key areas where neuromorphic computing is likely to have a significant impact include:
1. Robotics: Neuromorphic computing can provide real-time processing and learning, allowing robots to adapt and learn in complex and dynamic environments.
2. Autonomous vehicles: Neuromorphic computing can provide the real-time processing and learning required for autonomous vehicles to navigate complex and dynamic environments.
3. Smart homes: Neuromorphic computing can provide the real-time processing and learning required for smart homes to adapt and learn the preferences and habits of their occupants.
4. Healthcare: Neuromorphic computing can provide the real-time processing and learning required for medical devices to diagnose and treat complex medical conditions.

### Sources
- [1] https://www.mitpressjournals.org/doi/abs/10.1162/neco_a_01325 
- [2] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4087181/ 
- [3] https://ieeexplore.ieee.org/abstract/document/7551405 
- [4] https://www.nature.com/articles/s41598-019-40737-7 

## Technical Overview
Neuromorphic computing is a complex and multidisciplinary field that involves the development of computer systems that mimic the structure and function of the human brain. The technical overview of neuromorphic computing involves the development of artificial neural networks, memristors, spintronics, and neuromorphic chips. These technologies are designed to provide a highly efficient and adaptive way of processing information, allowing for real-time learning and processing.

### Artificial Neural Networks
Artificial neural networks are a key component of neuromorphic computing systems. They are designed to mimic the structure and function of the brain's neural networks, providing a highly efficient and adaptive way of processing information. Artificial neural networks consist of layers of interconnected nodes or "neurons," which process and transmit information in a parallel and adaptive manner.

### Memristors
Memristors are a type of memory device that can store data in a non-volatile manner, allowing for more efficient and adaptive processing. They are designed to mimic the synapses of the brain, providing a highly efficient and scalable way of processing data. Memristors have a number of advantages over traditional memory devices, including low power consumption, high density, and fast switching speeds.

### Spintronics
Spintronics is a technology that uses the spin of electrons to process information, providing a more efficient and scalable way of processing data. Spintronics is designed to mimic the spin-based processing of the brain, allowing for highly efficient and adaptive processing. Spintronics has a number of advantages over traditional computing technologies, including low power consumption, high density, and fast switching speeds.

## Applications and Future Directions
Neuromorphic computing has a number of potential applications, including robotics, autonomous vehicles, smart homes, and healthcare. These applications require real-time processing and learning, making neuromorphic computing an ideal solution. The future directions of neuromorphic computing involve the development of more advanced artificial neural networks, memristors, spintronics, and neuromorphic chips.

### Robotics
Neuromorphic computing can provide real-time processing and learning, allowing robots to adapt and learn in complex and dynamic environments. This makes neuromorphic computing an ideal solution for robotics applications, including industrial robotics, service robotics, and autonomous robots.

### Autonomous Vehicles
Neuromorphic computing can provide the real-time processing and learning required for autonomous vehicles to navigate complex and dynamic environments. This makes neuromorphic computing an ideal solution for autonomous vehicle applications, including self-driving cars, drones, and other autonomous systems.

### Smart Homes
Neuromorphic computing can provide the real-time processing and learning required for smart homes to adapt and learn the preferences and habits of their occupants. This makes neuromorphic computing an ideal solution for smart home applications, including home automation, energy management, and security systems.

## Conclusion
Neuromorphic computing is an emerging field that involves the development of computer systems that mimic the structure and function of the human brain. It has a number of potential applications, including robotics, autonomous vehicles, smart homes, and healthcare. The technical overview of neuromorphic computing involves the development of artificial neural networks, memristors, spintronics, and neuromorphic chips. These technologies are designed to provide a highly efficient and adaptive way of processing information, allowing for real-time learning and processing. The future directions of neuromorphic computing involve the development of more advanced artificial neural networks, memristors, spintronics, and neuromorphic chips.

In [29]:
prompt_template="""
Write a detailed technical report on the mathematical topic: {topic}.

Structure the report into the following sections, each clearly separated with appropriate Markdown headings (#, ##):

# Introduction

{{Give a clear, concise overview of the topic.}}

{{Explain its significance and typical applications.}}

# Mathematical Theory

{{Describe the core mathematical foundations in detail.}}

{{Include important definitions, theorems, proofs, and examples where relevant.}}

# Latest Open Problems

{{Discuss major current research challenges or unresolved questions in the field.}}

# Reference recent papers or developments (with citations).

# Future Directions

{{Suggest possible paths for future research or applications.}}

{{Discuss emerging trends and technologies that might impact this area.}}

# References

List all sources cited, preferably in a simple numbered list or BibTeX-style.

Use appropriate LaTeX-style formatting (with $$ for equations if needed) for mathematical expressions inside the Markdown.
Write in a formal and precise tone, suitable for a technical audience.
"""


response = model.invoke(prompt_template.format(topic="Discrete Mathematics"))
from IPython.display import Markdown
Markdown(response.content)

# Introduction
Discrete mathematics is a branch of mathematics that deals with mathematical structures that are fundamentally discrete, in the sense that they are composed of distinct, individual elements, rather than being continuous. This field has gained significant importance in recent years due to its wide range of applications in computer science, cryptography, coding theory, and optimization problems. Discrete mathematics provides the mathematical foundations for many computational systems, including algorithms, data structures, and software design. Its significance lies in its ability to model and analyze complex systems, making it a crucial tool for problem-solving in various fields.

Discrete mathematics has numerous applications, including computer networks, database systems, and coding theory. For instance, graph theory, a subfield of discrete mathematics, is used to model and analyze network structures, such as social networks, transportation systems, and communication networks. Additionally, discrete mathematics is essential in cryptography, as it provides the mathematical foundations for secure data transmission and encryption methods.

## Significance and Applications
The significance of discrete mathematics can be seen in its ability to provide a framework for solving complex problems in various fields. Its applications include:

* Computer science: discrete mathematics provides the mathematical foundations for algorithms, data structures, and software design.
* Cryptography: discrete mathematics is used to develop secure encryption methods and cryptographic protocols.
* Coding theory: discrete mathematics is used to develop error-correcting codes and data compression algorithms.
* Optimization problems: discrete mathematics is used to solve optimization problems, such as scheduling, resource allocation, and network flow problems.

# Mathematical Theory
The core mathematical foundations of discrete mathematics include set theory, graph theory, number theory, and combinatorics. These areas provide the mathematical framework for analyzing and solving problems in discrete mathematics.

## Set Theory
Set theory is a fundamental area of discrete mathematics that deals with the study of sets, which are collections of distinct objects. Set theory provides the mathematical foundations for many areas of discrete mathematics, including graph theory and combinatorics. The basic concepts in set theory include:

* Sets: a collection of distinct objects.
* Subsets: a set whose elements are all elements of another set.
* Union: the set of all elements that are in at least one of two sets.
* Intersection: the set of all elements that are in both of two sets.

The set operations can be represented using the following equations:
$$A \cup B = \{x \mid x \in A \text{ or } x \in B\}$$
$$A \cap B = \{x \mid x \in A \text{ and } x \in B\}$$

## Graph Theory
Graph theory is a subfield of discrete mathematics that deals with the study of graphs, which are collections of vertices connected by edges. Graph theory provides the mathematical framework for modeling and analyzing network structures, such as social networks, transportation systems, and communication networks. The basic concepts in graph theory include:

* Graphs: a collection of vertices connected by edges.
* Adjacency matrix: a matrix that represents the connections between vertices in a graph.
* Graph isomorphism: two graphs are isomorphic if they have the same structure.

The adjacency matrix of a graph can be represented using the following equation:
$$A = \begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn}
\end{bmatrix}$$
where $a_{ij} = 1$ if there is an edge between vertices $i$ and $j$, and $a_{ij} = 0$ otherwise.

## Number Theory
Number theory is a subfield of discrete mathematics that deals with the study of properties of integers and other whole numbers. Number theory provides the mathematical foundations for many areas of discrete mathematics, including cryptography and coding theory. The basic concepts in number theory include:

* Divisibility: an integer $a$ is divisible by an integer $b$ if there exists an integer $c$ such that $a = bc$.
* Primality: an integer $p$ is prime if it is divisible only by $1$ and itself.
* Congruences: two integers $a$ and $b$ are congruent modulo $n$ if $a \equiv b \pmod{n}$.

The congruence relation can be represented using the following equation:
$$a \equiv b \pmod{n} \iff a - b = kn$$
for some integer $k$.

## Combinatorics
Combinatorics is a subfield of discrete mathematics that deals with the study of counting and arranging objects in various ways. Combinatorics provides the mathematical framework for solving problems in many areas of discrete mathematics, including graph theory and number theory. The basic concepts in combinatorics include:

* Permutations: a permutation of a set of objects is an arrangement of the objects in a specific order.
* Combinations: a combination of a set of objects is a selection of objects from the set, without regard to order.
* Recurrence relations: a recurrence relation is a mathematical equation that defines a sequence of numbers recursively.

The recurrence relation for the Fibonacci sequence can be represented using the following equation:
$$F(n) = F(n-1) + F(n-2)$$
for $n \geq 2$, with initial conditions $F(0) = 0$ and $F(1) = 1$.

# Latest Open Problems
There are many open problems in discrete mathematics, including:

* The P versus NP problem: this problem deals with the relationship between computational complexity and verifiability.
* The traveling salesman problem: this problem deals with finding the shortest possible tour that visits a set of cities and returns to the starting city.
* The knapsack problem: this problem deals with finding the optimal subset of items to include in a knapsack of limited capacity.

Recent papers on these topics include [1], [2], and [3]. These papers provide new insights and approaches to solving these problems, and demonstrate the ongoing research efforts in discrete mathematics.

# Future Directions
Discrete mathematics is a rapidly evolving field, with many emerging trends and technologies that are expected to impact this area. Some possible future directions for research in discrete mathematics include:

* Quantum computing: the development of quantum computers is expected to have a significant impact on discrete mathematics, particularly in areas such as cryptography and coding theory.
* Artificial intelligence: the development of artificial intelligence and machine learning algorithms is expected to have a significant impact on discrete mathematics, particularly in areas such as graph theory and combinatorics.
* Network science: the study of complex networks is expected to continue to be an active area of research in discrete mathematics, with applications in many fields, including social networks, transportation systems, and communication networks.

# References
1. S. Cook, "The P versus NP problem", Journal of Computer and System Sciences, vol. 77, no. 2, pp. 267-276, 2011.
2. M. Garey and D. Johnson, "Computers and Intractability: A Guide to the Theory of NP-Completeness", W.H. Freeman and Company, 1979.
3. D. Knuth, "The Art of Computer Programming, Volume 4A: Combinatorial Algorithms, Part 1", Addison-Wesley, 2011.